<a href="https://colab.research.google.com/github/kingsgameBen/MLbasic_0223/blob/main/mlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MLP Multi Layer Perceptron
# 多層感知器

### TensorFlow 2版 （把keras融合進來了）
Tensorflow 1.x版 import keras.xx ==> 現在 import tensorflow.keras.xxx

pycharm安裝tensorflow: \
 - pycharm3.8: 直接安裝最新版, 
 - pycharm3.7以前: 安裝2.0.0版

[https://keras.io/zh/ ] \
###Keras 
神經網路API,以TensorFlow、CNTK..作為後端來運行 \
開發重點是支持快速的實驗。以最小的延遲把想法轉換成實驗結果
核心數據結構是model


numpy用於多維陣列、讀取及數學運算處理。ndarray : n維數組容器

pandas 用於表格等資料預處理。Series、DataFrame

sklearn則是對欄位的資料做計算分析

matplotlib、seaborn資料視覺化

使用tuple型態承接 mnist資料集


In [ ]:
# 安裝資料集 - 明斯特資料集(手寫數字)
from tensorflow.keras.datasets.mnist import load_data
# ((x_train, y_train), (x_test, y_test))
# numpy array ---  m*n 的多維陣列
(x_train, y_train), (x_test, y_test) = load_data()


ndarray.shape
[https://numpy.org/doc/stable/reference/generated/numpy.ndarray.shape.html ]

In [ ]:
# 數組陣列的維度
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
#計算 輸入的訓練資料集 的維度 28px * 28px
print(x_train.shape[1]*x_train.shape[2])

(cmap參數：colormap)\
[https://matplotlib.org/stable/tutorials/colors/colormaps.html ]

(imshow)\
[https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.imshow.html ]


In [ ]:
import matplotlib.pyplot as plt
# 顯示數字圖片
# matplotlib colormap  關鍵參數cmap
# plt.imshow(x_train[0], cmap="PiYG")   # pink~yellowgreen
# plt.imshow(x_train[0], cmap="PiYG_r") # 反向
# plt.imshow(x_train[0], cmap="gray"
plt.imshow(x_train[0], cmap="bwr")

In [ ]:
import pandas as pd
pd.DataFrame(x_train[15])

# Sequential 建立模型

### Dense 稠密層,全連接層: 每一根神經都跟前面的輸入相連
(layers)[ https://keras.io/zh/layers/core/ ]

(Sequential)[https://keras.io/zh/models/sequential/#sequential-api ] \

( 習慣上會抓784總數除以10=>78個左右, 再以2的倍數抓78的上下範圍去try第一層要輸出多少個\
ex: 2, 4, 8...64, 128, 256, 512...  先猜個128個, 跑出分數後再回來調整try ) \

第1層連接層:(輸出神經元256個,激勵函數:relu,輸入神經元784個)

第2層連接層:(輸出神經元10個, 激勵函數softmax)

Sequential的 summary():顯示目前網路架構

Param:參數, 每層神經元的權重個數weights

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
total_train_size = x_train.shape[1]*x_train.shape[2]
layers = [
        # 64, 128, 256, 512..
        #   784 * 128 + 128(bias)
        # 測試最終採用256 => 784 * 256 + 256(bias)
          Dense(256, activation="relu", input_dim=total_train_size),
        #   128 * 10 + 10(bias)
          Dense(10, activation="softmax")
]
model = Sequential(layers)
model.summary()
# 調整參數
# 128 * 10 = 1280, 1280 + 10(bias)

# 2. 給出Loss

如果只有一個神經元時(兩個機率)：BinaryCrossEntropy \
多個神經元：pilog(1/qi): CategoricalCrossEntropy \
$$P_ilog(\frac{1}{q_i})$$

compile():用於配置訓練模型

In [ ]:
# 如果只有一個神經元時(兩個機率)：BinaryCrossEntropy
# 多個神經元：pilog(1/qi): CategoricalCrossentropy
from tensorflow.keras.losses import CategoricalCrossentropy
model.compile(loss=CategoricalCrossentropy(),
              metrics=["accuracy"],
              optimizer="adam")

One-Hot encoding \
[https://www.tensorflow.org/api_docs/python/tf/keras/utils/to_categorical ] \
num_classes 強迫劃分成10個欄位 \



In [ ]:
from tensorflow.keras.utils import to_categorical
y_train_cat = to_categorical(y_train, 10)
y_test_cat = to_categorical(y_test, 10)
print(y_train[0])
print(y_train_cat[0])

# 訓練模型

分佈區間很重要,希望落點在靠中間一點的區域

0~1之間

除以255是為了把區間縮減, 圖像辨識是因為RGB的取值範圍在0~255

In [ ]:
# reshape可以偷懶不寫其中一個參數, 設-1時, 他會自己找有多少筆  (train:60000, test:10000)
x_train_shape = x_train.reshape(-1, 784) / 255.0
x_test_shape = x_test.reshape(-1, 784)  / 255.0
# print(x_train_shape[12345])


[https://keras.io/zh/callbacks/ ]

EarlyStopping

小批次梯度下降\
[https://keras-cn.readthedocs.io/en/latest/legacy/getting_started/concepts/ ]

SGD參數：是基於mini-batch 的stochastic gradient descent 隨機梯度下降的縮寫

patience:當訓練出的loss不再進步,我還要保持耐心等你繼續訓練幾個epochs

restore_best_weights:True取每輪訓練裡成績最好的那個, Fasle則取最後一輪的訓練成績

verbose: 0, 1 或 2。日志显示模式。 0 = 安静模式, 1 = 进度条, 2 = 每轮一行

In [ ]:
# validation_split 切一小段資料驗證60000*0.1=6000筆不訓練,拿來驗證訓練
# 整份資料54000筆一次
# 控制log紀錄顯示方式 verbose 0, 1, 2 => 2代表不要顯示進度條
# keras.callback 回調函數  每一輪epoch就回調一次
from tensorflow.keras.callbacks import EarlyStopping
callbacks = [
             EarlyStopping(patience=5, restore_best_weights=True)
]
model.fit(x_train_shape,
          y_train_cat,
          batch_size=200,
          epochs=50,
          validation_split=0.1,
          callbacks=callbacks,
          verbose=2)
# 200筆一調, 調整270次
# 看 val_loss  用機率來看

In [ ]:
# train - validate - test
model.evaluate(x_test_shape, y_test_cat)

In [ ]:
from sklearn.metrics import confusion_matrix
# sklearn:    predict_proba/ predict
# tensorflow: predict      / predict_classes
pre = model.predict_classes(x_test_shape)
mat = confusion_matrix(y_test, pre)
pd.DataFrame(mat,
             columns=[ str(i) + "(預測)" for i in range(10)],
             index=[ str(i) + "(正確)" for i in range(10)])

In [ ]:
import numpy as np
idx = np.nonzero(pre != y_test)[0]
print(idx)
print(len(idx))
false_img = x_test[idx]
false_pre = pre[idx]
false_label = y_test[idx]
# print(false_img)
# print(false_pre)
# print(false_label)
# subplot(高度,寬度，ith)
plt.figure(figsize=(14, 42))
width = 10
height = len(idx) // width + 1
for i in range(len(idx)):
    plt.subplot(height, width, i+1)
    plt.imshow(false_img[i])
    t = "[Ori]:{}\n[Pre]:{}".format(false_label[i], false_pre[i])
    plt.title(t)
    plt.axis("off")

In [ ]:
# 先看row, 再看column
mat_sample = [
               [1, 0, 0],
               [0, 2, 0],
               [1, 0, 0],
               [0, 3, 5]
]
np.nonzero(mat_sample)

In [ ]:
# Pillow
from PIL import Image
# RGB, RGBA, L(gray)
# img = Image.open("test.png").resize((28, 28)).convert("L")
# img_np = np.array(img)
# img_np_r = img_np.reshape(1, 784)
# proba = model.predict(img_np_r)
# print(proba)
# plt.imshow(img_np)
fn = input("輸入檔名:")
img = Image.open(fn).resize((28, 28)).convert("L")
img_np = np.array(img)
img_np_r = img_np.reshape(1, 784) / 255.0
proba = model.predict(img_np_r)[0]
for i in range(10):
    print(i, "的機率:", proba[i])
ans = model.predict_classes(img_np_r)[0]
print("答案:", ans)
plt.imshow(img_np)